# Quantum Facts in Qiskit
Here some useful topics will be covered and others recalled to make you more familiar with quantum terms and concepts.

Start by some typical setup and definition of useful functions, which you are encouraged to look at.

Then, head to the [exercises start](#Exercises-Start-Here) to start coding!

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute

In [ ]:
# Choose the drawer you like best:
from qiskit.tools.visualization import matplotlib_circuit_drawer as draw
#from qiskit.tools.visualization import circuit_drawer as draw

In [ ]:
from qiskit import IBMQ
IBMQ.load_accounts() # make sure you have setup your token locally to use this

In [ ]:
%matplotlib inline

## Utils for visualizing experimental results

In [ ]:
import matplotlib.pyplot as plt

def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

## Utils for executing circuits

In [ ]:
from qiskit import Aer
# See a list of available local simulators
print("Aer backends: ", Aer.backends())

In [ ]:
# see a list of available remote backends (these are freely given by IBM)
print("IBMQ Backends: ", IBMQ.backends())

### Execute locally

In [ ]:
# execute circuit and either display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: ", result_sim, result_counts)
    
    if draw_circuit: # draw the circuit
        draw(qc)
    else: # or show the results
        show_results(result_counts)

### Execute remotely

In [ ]:
from qiskit.backends.ibmq import least_busy
import time
# Compile and run on a real device backend
def execute_remotely(qc, draw_circuit=False):
    if draw_circuit: # draw the circuit
        draw(qc)
    try:
        # select least busy available device and execute.
        least_busy_device = least_busy(IBMQ.backends(simulator=False))
        print("Running on current least busy device: ", least_busy_device)

        # running the job
        job_exp = execute(qc, backend=least_busy_device, shots=1024, max_credits=10)

        lapse, interval = 0, 10
        while job_exp.status().name != 'DONE':
            print('Status @ {} seconds'.format(interval * lapse))
            print(job_exp.status())
            time.sleep(interval)
            lapse += 1
        print(job_exp.status())
        exp_result = job_exp.result()
        result_counts = exp_result.get_counts(qc)

        # Show the results
        print("experiment: ", exp_result, result_counts)
        if not draw_circuit: # show the results
            show_results(result_counts)
    except:
        print("All devices are currently unavailable.")

## Building the circuit

In [ ]:
def new_circuit(size):
    # Create a Quantum Register with size qubits
    qr = QuantumRegister(size)

    # Create a Classical Register with size bits
    cr = ClassicalRegister(size)

    # Create a Quantum Circuit acting on the qr and cr register
    return qr, cr, QuantumCircuit(qr, cr)

---
<h1 align="center">Exercises Start Here</h1>

Make sure you ran all the above cells in order, as the following exercises use functions defined and imported above.

## Creating Entanglement
Although, you have already entangled qubits on the previous week, you were not aware of it.

Naturally, we will redo some of those tasks and focus on interpreting their results.

### Hadamard + CNOT

**TASK:** Create a new circuit with 2 qubits using `new_circuit` (very useful to reconstruct your circuit in Jupyter)

In [ ]:
# create the circuit


**TASK:** Add a Hadamard on qubit 0

In [ ]:
# H gate on qubit 0

**TASK:** Add a CX (CNOT) gate on control qubit 0 and target qubit 1

**TASK:** Perform a measurement

In [ ]:
# measure the qubits


**TASK:** check the result using `execute_locally` test both `True` and `False` for the `draw_circuit` option

The result should be something like `COMPLETED {'00': 506, '11': 518}`.

**TASK:** What does this mean?

## Bell state in IBMQ
**TASK:** Get the QASM specification for your code and use it on [IBMQ QASM editor](https://quantumexperience.ng.bluemix.net/qx/qasm)

---
### More entanglement
**TASK:** Repeat the previous circuit, but add a bit-flip on the target qubit

In [ ]:
# create the circuit


**TASK:** Add a Hadamard on qubit 0

In [ ]:
# H gate on qubit 0


**TASK:** Add an X gate on qubit 1

In [ ]:
# X gate on qubit 1


**TASK:** Add a CX (CNOT) gate on control qubit 0 and target qubit 1

**TASK:** Perform a measurement

In [ ]:
# measure the qubits


**TASK:** check the result using `execute_locally` test both `True` and `False` for the `draw_circuit` option

**TASK:** Observe the results

## Quantum measurement in different basis
Given any of our previous circuits, we are going to measure them on both the zero-one basis as well as on the plus-minus basis.

**TASK:** Reconstruct one of the previous circuits (do not apply the measurement yet)

In [ ]:
# create the circuit

**TASK:** Add a Hadamard after each qubit

**TASK:** perform the measurement now and execute

**TASK:** Why is this measurement different from the previous one?

# Free flow
Take a look at the following explanation of Entanglement, taken from the [qiskit-tutorial](https://github.com/Qiskit/qiskit-tutorial/blob/master/community/terra/qis_intro/entanglement_introduction.ipynb)

### Entanglement

The core idea behind the second Principle is *entanglement*. Upon reading the Principle, one might be inclined to think that entanglement is simply strong correlation between two entitities -- but entanglement goes well beyond mere perfect (classical) correlation. If you and I read the same paper, we will have learned the same information. If a third person comes along and reads the same paper they <i>also</i> will have learned this information. All three persons in this case are perfectly correlated, and they will remain correlated even if they are separated from each other. 

The situation with quantum entanglement is a bit more subtle. In the quantum world, you and I could read the same quantum paper, and yet we will not learn what information is actually contained in the paper until we get together and share our information. However, when we are together, we find that we can unlock more information from the paper than we initially thought possible. Thus, quantum entanglement goes much further than perfect correlation.

To demonstrate this, we will define the controlled-NOT (CNOT) gate and the composition of two systems. The convention we use Qiskit is to label states by writing the first qubit's name in the rightmost position, thereby allowing us to easily convert from binary to decimal. As a result, we define the tensor product between operators $q_0$ and $q_1$ by $q_1\otimes q_0$. 

Taking $q_0$ as the control and $q_1$ as the target, the CNOT with this representation is given by

$$ CNOT =\begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 0 & 0 & 1\\0& 0& 1 & 0\\0 & 1 & 0 & 0 \end{pmatrix},$$

which is non-standard in the quantum community, but more easily connects to classical computing, where the least significant bit (LSB) is typically on the right. An entangled state of the two qubits can be made via an $H$ gate on the control qubit, followed by the CNOT gate. This generates a particular maximally entangled two-qubit state known as a Bell state, named after John Stewart Bell ([learn more about Bell and his contributions to quantum physics and entanglement](https://en.wikipedia.org/wiki/John_Stewart_Bell)). 


---

# Entanglement on a real device
**TASK:** Create a simple entanglement and execute it on a real device.

In [ ]:
execute_remotely(circuit)

**TASK:** Comment on the results